## Environment Setup

In [ ]:
! pip install langchain_community langchain_mistralai langchainhub langchain tiktoken chromadb

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '<your LangChain API KEY goes here>'
os.environ['MISTRAL_API_KEY'] = '<your Mistral API KEY goes here>'
os.environ['HF_TOKEN'] = '<your Hugging Face TOKEN goes here>'

In [ ]:
questions = [ "Who is the main character in Moby Dick?",
              "Who is Ishmael, and why is he important?",
              "What role does Queequeg play in Moby Dick?"]

## Indexing

### Load the text

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
# there can be mulitple urls
loader = WebBaseLoader("https://www.gutenberg.org/cache/epub/2701/pg2701.txt")
books = loader.load()

### Splitting the text

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(books)
len(splits)

### Indexing to ChromaDB

In [ ]:
from langchain_mistralai import MistralAIEmbeddings
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=MistralAIEmbeddings())


## Retrieval

### Retrieval of relevant documents

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
docs = retriever.get_relevant_documents(questions[0])
len(docs)
docs[0]

# Generation

## Initializing the Prompt template for Generator

In [ ]:
from langchain_mistralai import ChatMistralAI
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

## Init LLM

In [ ]:
# LLM
llm = ChatMistralAI(model_name="mistral-large-latest", temperature=0)

## Init simple Chain

In [ ]:
# Chain
chain = prompt | llm

## Invoking the trivial chain for a given user question

In [ ]:
# Run
chain.invoke({"context":docs,"question":questions[0]})

## Preparing and calling RAG chain (with parameter, parser)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
for question in questions:
    print("-------------")
    print(question)
    print(rag_chain.invoke(question))